<a href="https://colab.research.google.com/github/venkatrajan247/Citizen-Science-EnviroTrack/blob/main/dpo_optimized_mistral_7b_finetuning.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

## Unsloth installation and importing libraries

In [ ]:
%%capture
!pip install pip3-autoremove
!pip-autoremove torch torchvision torchaudio -y
!pip install "torch==2.4.0" "xformers==0.0.27.post2" triton torchvision torchaudio
!pip install "unsloth[kaggle-new] @ git+https://github.com/unslothai/unsloth.git"

In [ ]:
import os
os.environ["WANDB_DISABLED"] = "true"

In [ ]:
import torch
from unsloth import PatchDPOTrainer
from transformers import TrainingArguments
from trl import DPOTrainer
from unsloth import FastLanguageModel
from datasets import load_dataset
PatchDPOTrainer()

🦥 Unsloth: Will patch your computer to enable 2x faster free finetuning.
🦥 Unsloth Zoo will now patch everything to make training faster!


Using the `WANDB_DISABLED` environment variable is deprecated and will be removed in v5. Use the --report_to flag to control the integrations used for logging result (for instance --report_to none).
Using the `WANDB_DISABLED` environment variable is deprecated and will be removed in v5. Use the --report_to flag to control the integrations used for logging result (for instance --report_to none).
Using the `WANDB_DISABLED` environment variable is deprecated and will be removed in v5. Use the --report_to flag to control the integrations used for logging result (for instance --report_to none).
Using the `WANDB_DISABLED` environment variable is deprecated and will be removed in v5. Use the --report_to flag to control the integrations used for logging result (for instance --report_to none).
Using the `WANDB_DISABLED` environment variable is deprecated and will be removed in v5. Use the --report_to flag to control the integrations used for logging result (for instance --report_to none).
Using

## Loading dataset

In [ ]:
# Load the dataset from the specified Hugging Face repository and split it into the 'train' set
dataset = load_dataset("Chirag4579/security-dpo-clean", split='train')

# Rename the 'text' column to 'chosen' for clarity and to align with the expected column naming
dataset = dataset.rename_column('text', 'chosen')

# Rename the 'rejected_text' column to 'rejected' for consistency with the 'chosen' column
dataset = dataset.rename_column('rejected_text', 'rejected')

data.jsonl:   0%|          | 0.00/192M [00:00<?, ?B/s]

Generating train split:   0%|          | 0/244930 [00:00<?, ? examples/s]

In [ ]:
dataset

Dataset({
    features: ['prompt', 'chosen', 'rejected'],
    num_rows: 244930
})

In [ ]:
# In the example we are taking a small sample of data as it will take 1.5 to 2 hours to finetune

# Split the dataset into training and testing sets, with 40% of the data allocated to the test set
dataset = dataset.train_test_split(test_size=0.4)

# Select the first 6000 samples from the training set for further processing
dataset['train'] = dataset['train'].select(range(6000))

# Select the first 2000 samples from the testing set for further processing
dataset['test'] = dataset['test'].select(range(2000))

In [ ]:
dataset

DatasetDict({
    train: Dataset({
        features: ['prompt', 'chosen', 'rejected'],
        num_rows: 6000
    })
    test: Dataset({
        features: ['prompt', 'chosen', 'rejected'],
        num_rows: 2000
    })
})

## Loading model and its configurations

In [ ]:
# Define the maximum sequence length for the model to process
max_seq_length = 4096

# Specify the data type for the model (set to None for default behavior)
dtype = None

# Enable 4-bit quantization to reduce memory usage; set to False if higher precision is required
load_in_4bit = True

# Load the pretrained language model and tokenizer using the specified parameters
model, tokenizer = FastLanguageModel.from_pretrained(
    model_name="unsloth/mistral-7b-instruct-v0.3-bnb-4bit",  # Model name from the Hugging Face hub
    max_seq_length=max_seq_length,  # Maximum token limit for input sequences
    dtype=dtype,  # Data type for model parameters
    load_in_4bit=load_in_4bit,  # Flag for enabling 4-bit quantization
)

==((====))==  Unsloth 2024.12.8: Fast Mistral patching. Transformers: 4.46.3.
   \\   /|    GPU: Tesla T4. Max memory: 14.741 GB. Platform: Linux.
O^O/ \_/ \    Torch: 2.4.0+cu121. CUDA: 7.5. CUDA Toolkit: 12.1. Triton: 3.0.0
\        /    Bfloat16 = FALSE. FA [Xformers = 0.0.27.post2. FA2 = False]
 "-____-"     Free Apache license: http://github.com/unslothai/unsloth
Unsloth: Fast downloading is enabled - ignore downloading bars which are red colored!


model.safetensors:   0%|          | 0.00/4.14G [00:00<?, ?B/s]

generation_config.json:   0%|          | 0.00/157 [00:00<?, ?B/s]

tokenizer_config.json:   0%|          | 0.00/141k [00:00<?, ?B/s]

tokenizer.model:   0%|          | 0.00/587k [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/446 [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/1.96M [00:00<?, ?B/s]

In [ ]:
# Apply PEFT (Parameter-Efficient Fine-Tuning) to the quantized model using LoRA
model = FastLanguageModel.get_peft_model(
    model,  # The pre-trained and potentially quantized model

    # The rank of the low-rank decomposition matrices for LoRA. Higher values allow
    # the model to capture more task-specific information (common values: 8, 16, 32, 64, 128).
    r=64,

    # Specify the target modules where LoRA should be applied, such as key, value,
    # query projections, and other attention-related layers.
    target_modules=[
        "q_proj", "k_proj", "v_proj", "o_proj",  # Attention projection layers
        "gate_proj", "up_proj", "down_proj",    # Feed-forward network layers
    ],

    # LoRA-specific hyperparameters:
    lora_alpha=64,  # Scaling factor for the LoRA update
    lora_dropout=0,  # Dropout for LoRA layers; set to 0 for no dropout
    bias="none",  # Bias handling strategy; options are "none", "all", or "lora_only"

    # Enable gradient checkpointing for memory efficiency during training.
    # "unsloth" is a custom value, likely to support longer context lengths.
    use_gradient_checkpointing="unsloth",

    random_state=3407,  # Random seed for reproducibility during training

    # Optional configurations for advanced techniques:
    use_rslora=False,  # Flag for whether to use Randomized Singular LoRA (RsLoRA)
    loftq_config=None,  # Configuration for LoFT-Q (LoRA for Fine-Tuning Quantized models)
)


Unsloth 2024.12.8 patched 32 layers with 32 QKV layers, 32 O layers and 32 MLP layers.


In [ ]:
# Initialize the DPOTrainer (Direct Preference Optimization Trainer) to fine-tune the model
dpo_trainer = DPOTrainer(
    model=model,  # The model to be fine-tuned (already loaded and potentially quantized with LoRA)

    # Define training arguments for fine-tuning
    args=TrainingArguments(
        per_device_train_batch_size=2,  # Number of samples processed per device in one forward/backward pass
        gradient_accumulation_steps=3,  # Accumulate gradients over multiple steps to simulate a larger batch size
        warmup_ratio=0.1,  # Fraction of total steps used for learning rate warmup
        num_train_epochs=1,  # Number of training epochs
        learning_rate=5e-6,  # Initial learning rate for the optimizer
        fp16=not torch.cuda.is_bf16_supported(),  # Use FP16 if BF16 is not supported
        bf16=torch.cuda.is_bf16_supported(),  # Use BF16 precision if supported by the GPU
        logging_steps=10,  # Log training progress every 10 steps
        optim="adamw_8bit",  # Use AdamW optimizer with 8-bit precision for memory efficiency
        weight_decay=0.001,  # Weight decay for regularization
        lr_scheduler_type="linear",  # Use a linear learning rate scheduler
        seed=42,  # Set random seed for reproducibility
        report_to="none",  # Disable reporting to external tools (e.g., WandB or TensorBoard)
        output_dir="outputs",  # Directory to save the model and training artifacts
    ),

    beta=0.1,  # Beta parameter for DPO; balances the loss for optimization
    train_dataset=dataset["train"],  # Training dataset
    eval_dataset=dataset["test"],  # Evaluation dataset
    tokenizer=tokenizer,  # Tokenizer corresponding to the model
    max_length=4096,  # Maximum token length for input sequences
    max_prompt_length=512,  # Maximum token length for prompts within input sequences
)

Extracting prompt from train dataset:   0%|          | 0/6000 [00:00<?, ? examples/s]

Applying chat template to train dataset:   0%|          | 0/6000 [00:00<?, ? examples/s]

Extracting prompt from eval dataset:   0%|          | 0/2000 [00:00<?, ? examples/s]

Applying chat template to eval dataset:   0%|          | 0/2000 [00:00<?, ? examples/s]

Tokenizing train dataset:   0%|          | 0/6000 [00:00<?, ? examples/s]

Tokenizing eval dataset:   0%|          | 0/2000 [00:00<?, ? examples/s]

## Starting to finetune

In [ ]:
dpo_trainer.train() #start the training

==((====))==  Unsloth - 2x faster free finetuning | Num GPUs = 1
   \\   /|    Num examples = 6,000 | Num Epochs = 1
O^O/ \_/ \    Batch size per device = 2 | Gradient Accumulation steps = 3
\        /    Total batch size = 6 | Total steps = 1,000
 "-____-"     Number of trainable parameters = 167,772,160


Step,Training Loss,rewards / chosen,rewards / rejected,rewards / accuracies,rewards / margins,logps / rejected,logps / chosen,logits / rejected,logits / chosen
10,0.692700,0.000465,-0.000393,0.316667,0.000859,-134.829605,-121.619919,-3.326092,-3.480960
20,0.681800,-0.003948,-0.026922,0.900000,0.022973,-126.523643,-114.461624,-3.353951,-3.508379
30,0.644400,-0.009693,-0.111173,0.950000,0.101480,-132.801071,-119.489143,-3.382381,-3.491533
40,0.535000,-0.014261,-0.379619,0.983333,0.365358,-140.832291,-131.195267,-3.323389,-3.448073
50,0.445800,-0.036215,-0.700470,0.950000,0.664255,-140.845047,-124.485176,-3.335336,-3.451905
60,0.277500,0.036665,-1.316330,1.000000,1.352996,-131.727493,-118.275826,-3.370235,-3.532094
70,0.170600,-0.150441,-2.291286,0.983333,2.140845,-161.809845,-132.461578,-3.355913,-3.436999
80,0.106300,-0.309203,-3.960793,0.983333,3.651590,-162.089813,-117.010231,-3.333212,-3.462518
90,0.073900,-0.671395,-6.074956,0.966667,5.403560,-188.100784,-127.128067,-3.318185,-3.427528
100,0.035500,-0.958885,-7.825130,1.000000,6.866243,-211.326797,-127.347275,-3.333061,-3.436771


TrainOutput(global_step=1000, training_loss=0.05934671961939148, metrics={'train_runtime': 7451.279, 'train_samples_per_second': 0.805, 'train_steps_per_second': 0.134, 'total_flos': 0.0, 'train_loss': 0.05934671961939148, 'epoch': 1.0})

## Saving Models

In [ ]:
model.save_pretrained("mistral-security-lora-trained")

In [ ]:
# to push to huggingface (optional)
# from huggingface_hub import login
# from kaggle_secrets import UserSecretsClient
# user_secrets = UserSecretsClient()
# hf_token = user_secrets.get_secret("HuggingFace") # Fetching the Hugging Face token from the Kaggle Secret keys add on
# login(token = hf_token) # Logging into Hugging Face Hub to access models and other resources
# model.push_to_hub("mistral-security-lora-trained")

README.md:   0%|          | 0.00/609 [00:00<?, ?B/s]

  0%|          | 0/1 [00:00<?, ?it/s]

adapter_model.safetensors:   0%|          | 0.00/671M [00:00<?, ?B/s]

Saved model to https://huggingface.co/mistral-security-lora-trained


In [ ]:
from unsloth import FastLanguageModel
import transformers

In [ ]:
# Load the fine-tuned model and tokenizer from the specified directory
# The model is loaded with 4-bit quantization enabled for memory efficiency
model, tokenizer = FastLanguageModel.from_pretrained(
    model_name="mistral-security-lora-trained",  # Path to the saved fine-tuned model
    max_seq_length=4096,  # Set the maximum sequence length for processing inputs
    load_in_4bit=True,  # Enable 4-bit quantization to reduce memory usage
)

# Prepare the model for inference (set it to evaluation mode)
FastLanguageModel.for_inference(model)  # This configures the model for inference without further training

==((====))==  Unsloth 2024.12.8: Fast Mistral patching. Transformers: 4.46.3.
   \\   /|    GPU: Tesla T4. Max memory: 14.741 GB. Platform: Linux.
O^O/ \_/ \    Torch: 2.4.0+cu121. CUDA: 7.5. CUDA Toolkit: 12.1. Triton: 3.0.0
\        /    Bfloat16 = FALSE. FA [Xformers = 0.0.27.post2. FA2 = False]
 "-____-"     Free Apache license: http://github.com/unslothai/unsloth
Unsloth: Fast downloading is enabled - ignore downloading bars which are red colored!


PeftModelForCausalLM(
  (base_model): LoraModel(
    (model): MistralForCausalLM(
      (model): MistralModel(
        (embed_tokens): Embedding(32768, 4096, padding_idx=770)
        (layers): ModuleList(
          (0-31): 32 x MistralDecoderLayer(
            (self_attn): MistralAttention(
              (q_proj): lora.Linear4bit(
                (base_layer): Linear4bit(in_features=4096, out_features=4096, bias=False)
                (lora_dropout): ModuleDict(
                  (default): Identity()
                )
                (lora_A): ModuleDict(
                  (default): Linear(in_features=4096, out_features=64, bias=False)
                )
                (lora_B): ModuleDict(
                  (default): Linear(in_features=64, out_features=4096, bias=False)
                )
                (lora_embedding_A): ParameterDict()
                (lora_embedding_B): ParameterDict()
                (lora_magnitude_vector): ModuleDict()
              )
              (k_proj):

## Testing fine-tuned model

In [ ]:
def predict(input_prompt):
    # Define the conversation context for the model, including system and user roles
    messages = [
        {"role": "system", "content": "You are a helpful assistant that assists users to find the correct methods/approach for security within an organization."},
        {"role": "user", "content": input_prompt}  # The user's input prompt is added here
    ]

    # Format the input message into the appropriate structure for the model using the tokenizer
    text = tokenizer.apply_chat_template(
        messages,  # The structured message for the assistant
        tokenize=False,  # Don't tokenize yet, just prepare the chat template
        add_generation_prompt=True  # Add any additional generation prompt required by the model
    )

    # Tokenize the message text into model input format
    model_inputs = tokenizer([text], return_tensors="pt")

    # Generate model output by passing the tokenized input to the model
    # 'max_new_tokens' ensures that the model generates up to a maximum of 4096 new tokens
    generated_ids = model.generate(**model_inputs, max_new_tokens=4096,temperature=0.2)

    # Slice the generated sequence to remove the original input tokens (we only want the output)
    generated_ids = [output_ids[len(input_ids):] for input_ids, output_ids in zip(model_inputs.input_ids, generated_ids)]

    # Decode the generated token IDs back into a human-readable string, skipping special tokens
    response = tokenizer.batch_decode(generated_ids, skip_special_tokens=True)[0]

    # Return the generated response
    return response

In [ ]:
input_prompt = "What should be done when an employee leaves the organization?"
response = predict(input_prompt)
print(response)

1. Immediate Access Revocation: Revoke or disable the employee's access to all company systems, networks, and applications as soon as possible. This includes email accounts, cloud storage, and any other relevant platforms.

2. Data Transfer and Export: Ensure that the employee has transferred all necessary data to the appropriate individuals or departments before their departure. This may involve a data handover process to maintain business continuity.

3. Device Collection: Collect all company-issued devices, such as laptops, smartphones, and ID cards, to prevent unauthorized access or data leakage.

4. Account Deactivation: Deactivate or delete the employee's account in the company's HR system and any other relevant databases or directories.

5. Document and Record: Document the process of access revocation and device collection for auditing and compliance purposes.

6. Review and Update Access Controls: Review and update access controls to ensure that the departing employee's access

In [ ]:
input_prompt = "Explain the concept of log management in the context of SIEM."
response = predict(input_prompt)
print(response)

In the context of Security Information and Event Management (SIEM), Log Management plays a crucial role in monitoring and analyzing security-related events and activities within an organization's IT infrastructure.

1. Collection: SIEM systems collect log data from various sources such as firewalls, servers, applications, and network devices. This data provides a comprehensive view of the system's activities and behavior.

2. Normalization: The collected log data is then normalized to ensure consistency in format and structure, making it easier to compare and analyze across different systems and devices.

3. Correlation: SIEM systems use advanced algorithms to correlate related events and identify patterns or anomalies that may indicate a security threat or vulnerability. This helps in detecting potential security incidents and identifying potential risks.

4. Analysis: SIEM systems provide analytics and reporting capabilities to help security analysts investigate and understand the na